In [17]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from flask import Flask, jsonify,render_template
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from connections import password, client_id, client_secret
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
import requests
import pandas as pd
from keras.models import load_model
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [18]:
# Client credentials
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, 
    client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [19]:
top_model = load_model("Feature_Model_Trained.h5")

In [20]:
test_track = sp.audio_features('6fujklziTHa8uoM5OQSfIo')
track_df = pd.DataFrame(test_track, index = [0])
track_df.drop(columns = ['analysis_url', 'id', 'track_href', 'uri', 'type'], inplace = True)

In [21]:
track_df

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.142,0.794,291893,0.632,0,0,0.128,-6.163,1,0.0649,145.926,4,0.355


In [22]:
test_track[0]

{'danceability': 0.794,
 'energy': 0.632,
 'key': 0,
 'loudness': -6.163,
 'mode': 1,
 'speechiness': 0.0649,
 'acousticness': 0.142,
 'instrumentalness': 0,
 'liveness': 0.128,
 'valence': 0.355,
 'tempo': 145.926,
 'type': 'audio_features',
 'id': '6fujklziTHa8uoM5OQSfIo',
 'uri': 'spotify:track:6fujklziTHa8uoM5OQSfIo',
 'track_href': 'https://api.spotify.com/v1/tracks/6fujklziTHa8uoM5OQSfIo',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6fujklziTHa8uoM5OQSfIo',
 'duration_ms': 291893,
 'time_signature': 4}

In [23]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
X_scaler = StandardScaler().fit(track_df)
predict = X_scaler.transform(track_df)

In [24]:
top_model.predict_classes(predict)

array([0], dtype=int64)

In [25]:
# Establish SQL connection
connection_string = (f"root:{password}@localhost/spot_db")
engine = create_engine(f"mysql://{connection_string}")# , pool_recycle=3600, pool_pre_ping=True)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Save reference to the table
_features = Base.classes.features
_top = Base.classes.top_200_daily
# Create connection object
session = Session(engine)

In [56]:
features = pd.read_sql(session.query(_features).statement, session.bind)

X = features.drop(['index', 'ID'], axis = 1)
X.head(1)

,Acousticness,Danceability,Duration_ms,Energy,Instrumentalness,Key,Liveness,Loudness,Mode,Speechiness,Tempo,Time_Signature,Valence
0,0.131,0.748,188491,0.627,0.0,7,0.0852,-6.029,1,0.0644,120.963,4,0.524


In [57]:
features_unindex = features.drop('index', axis = 1)
features_unindex.to_csv('features.csv')

In [47]:
X_scaler = StandardScaler().fit(X)
predict = X_scaler.transform(X)

In [48]:
normal_predict = pd.DataFrame(predict)

In [52]:
# for feature in array:
results = top_model.predict_classes(x = X)
np.unique(results)

array([0], dtype=int64)

In [53]:
count_0 = 0
count_1 = 0
for result in results:
    if result == 0:
        count_0 += 1
    else:
        count_1 += 1
print(f" 0: {count_0} | 1: {count_1}")

 0: 11720 | 1: 0


In [42]:
X = X.iloc[0]